#### <u><font color="orange">**Notes:**</font></u>


<p>In this notebook, I will attempt to complete Exercise 2. It is intended to be readable by my teammates.</p> <br>

## <u>Exercise 2:</u>

<ol>
    <li>Implement RLS and (1+1) EA introduced in w3</li>
    <li>Run Random Search, RLS and (1+1) EA 10 times on the problems below.</li>
</ol>


## <u> Problems </u>

<p>Problems and functions will be used almost synonymously throughout this notebook. But more formally speaking, below are optimization problems, and the functions alone (starting with F) are not problems themselves, per sé (this should not be the first time we ask what a <i>problem</i> is -- think of decision problems in ADSA.). It is the idea or act of wanting to minimize or maximize them is the <b>problem.</b></p>

$$
\begin{array}{ll}
\text{F1 -- OM: } & \{0,1\}^n \to [0..n], x \mapsto \sum_{i=1}^{n}x_i \\
\text{F2 -- LO: } & \{0,1\}^n \to [0..n], x \mapsto \max{i \in [0..n] | \forall j\leq i: x_j = 1} = \sum_{i=1}^{n}\prod_{j=1}^{i} x_j \\
\text{F3 -- LHW: } & f:\{0,1\}^n\to \R, x \mapsto \sum_{i}ix_i \\
\text{F18 -- LABS: } & x\mapsto \frac{n^2}{2\sum_{k=1}^{n-1}(\sum_{i=1}^{n-k}s_i s_{i+k})^2}, \text{where } s_i = 2x_i - 1. \\
\text{F23 -- N-Queens Problem} & \\
\text{F24 -- Concentrated Trap} & \\
\text{F25 -- NK Landscapes (NKL)} &
\end{array}
$$


### **<u>Terminologies and Definitions</u>**


- `n` (int) := synonymous with the dimension of the problem or the problem size.
- `budget` (int) := (int) or the number of iterations or function evalutations. 

In [1]:
import os 
import numpy as np 
from shutil import rmtree 
import glob 

In [2]:
def clean():
    for name in ("my-experiment", "ioh_data"):
        for path in glob.glob(f"{name}*"):
            if os.path.isfile(path):
                os.remove(path)
            if os.path.isdir(path):
                rmtree(path, ignore_errors=True)


def ls(p="./"):
    for obj in os.listdir(os.path.normpath(p)):
        print(obj)


def cat(f):
    with open(os.path.normpath(f)) as h:
        print(h.read())


clean()

In [3]:
import ioh 
from ioh import ProblemClass, logger 

In [4]:
#  a list of problem can be accessed via the base classes 
real_problems: dict[int, str] = ioh.problem.RealSingleObjective.problems 
print(real_problems)

{1: 'Sphere', 2: 'Ellipsoid', 3: 'Rastrigin', 4: 'BuecheRastrigin', 5: 'LinearSlope', 6: 'AttractiveSector', 7: 'StepEllipsoid', 8: 'Rosenbrock', 9: 'RosenbrockRotated', 10: 'EllipsoidRotated', 11: 'Discus', 12: 'BentCigar', 13: 'SharpRidge', 14: 'DifferentPowers', 15: 'RastriginRotated', 16: 'Weierstrass', 17: 'Schaffers10', 18: 'Schaffers1000', 19: 'GriewankRosenbrock', 20: 'Schwefel', 21: 'Gallagher101', 22: 'Gallagher21', 23: 'Katsuura', 24: 'LunacekBiRastrigin', 30: 'UniformStarDiscrepancy10', 31: 'UniformStarDiscrepancy25', 32: 'UniformStarDiscrepancy50', 33: 'UniformStarDiscrepancy100', 34: 'UniformStarDiscrepancy150', 35: 'UniformStarDiscrepancy200', 36: 'UniformStarDiscrepancy250', 37: 'UniformStarDiscrepancy500', 38: 'UniformStarDiscrepancy750', 39: 'UniformStarDiscrepancy1000', 40: 'SobolStarDiscrepancy10', 41: 'SobolStarDiscrepancy25', 42: 'SobolStarDiscrepancy50', 43: 'SobolStarDiscrepancy100', 44: 'SobolStarDiscrepancy150', 45: 'SobolStarDiscrepancy200', 46: 'SobolStarDis

In [5]:
def compute_mean(X: np.ndarray) -> float:
    return float(np.mean(X))


def compute_sd(X: np.ndarray) -> float:
    return float(np.std(X))

In [6]:
def run_experiment(problem: ioh.problem.PBO, algorithm, num_runs = 10):
    """
    Run an experiment of a given algorithm on a given problem for a number of runs (budget)
    - `problem`: an ioh problem instance
    - `algorithm`: a callable that takes a problem as input and runs the algorithm on it
    - `num_runs`: number of independent runs to perform
    """
    # For a known problem 18 w/ 32 variables, we know optimum = 8
    if problem.meta_data.problem_id == 18 and problem.meta_data.n_variables == 32:
        optimum = 8
    else:
        optimum = problem.optimum.y # otherwise, calculate optimum???
    print(optimum)
    # =============================================================


    for run in range(num_runs):
        # run the algorithm on the problem 
        algorithm(problem)

        # print the best found for this run
        print(f"run: {run+1} - best found: {problem.state.current_best.y: .3f}")

        # reset the problem 
        problem.reset()

### <u>Let's begin implementing our algorithms</u>

In [7]:
# implement randomized local search 
class RandomizedLocalSearch:
    def __init__(self, budget: int, name: str = "RLS", algorithm_info: str = "Randomized Local Search"):
        self.budget = budget
        self.name = name
        self.algorithm_info = algorithm_info

    def __call__(self, problem: ioh.problem.PBO) -> None:
        # initialize a random solution 
        current = np.random.randint(0, 2, size=problem.meta_data.n_variables)
        current_value = problem(current.tolist())
        evaluations = 1

        while evaluations <= self.budget:
            # create a neighbor by flipping one random bit
            neighbor = current.copy()
            flip_index = np.random.randint(0, problem.meta_data.n_variables)
            neighbor[flip_index] = 1 - neighbor[flip_index] # flip the bit
            neighbor_value = problem(neighbor)
            evaluations += 1

            # if the neighbor is better or equal, move to the neighbor
            if neighbor_value >= current_value:
                current, current_value = neighbor, neighbor_value

In [8]:
# implement (1+1) EA
class OnePlusOneEA:
    def __init__(self, budget: int, name: str = "OnePlusOneEA", algorithm_info: str = "(1+1) Evolutionary Algorithm"):
        self.budget = budget
        self.name = name
        self.algorithm_info = algorithm_info

    def __call__(self, problem: ioh.problem.PBO) -> None:
        # initialize a random solution 
        current = np.random.randint(0, 2, size=problem.meta_data.n_variables)
        current_value = problem(current.tolist())
        evaluations = 1

        while evaluations <= self.budget:
            # create an offspring by flipping each bit with probability 1/n
            offspring = current.copy()
            for i in range(problem.meta_data.n_variables):
                if np.random.rand() < 1 / problem.meta_data.n_variables:
                    offspring[i] = 1 - offspring[i] # flip the bit
            offspring_value = problem(offspring)
            evaluations += 1

            # if the offspring is better or equal, move to the offspring
            if offspring_value >= current_value:
                current, current_value = offspring, offspring_value

In [7]:
# random search class 
class RandomSearch:
    def __init__(self, budget: int, name: str = "RandomSearch", algorithm_info: str = "Random Search Algorithm") -> None:
        """
        - `budget`: (int) number of function evaluations to perform
        """
        self.budget = budget
        self.name = name
        self.algorithm_info = algorithm_info

    def __call__(self, problem_func: ioh.problem.PBO) -> None:
        """
        - `problem`: an ioh problem instance
        """
        for _ in range(self.budget):
            # sample a random solution 
            X: np.ndarray = np.random.randint(2, size=problem_func.meta_data.n_variables) # randomized binary solution: a vector of 0s and 1s of length n_variables 
            # evaluate the solution 
            problem_func(X.tolist()) # unfortunately, ioh does not accept numpy arrays as input so we need to convert it to a list


#### Gather the required problems 

In [10]:
n = problem_size = 100 

In [9]:
p1 = ioh.get_problem(fid=1, 
                    dimension=n,
                    instance=1,
                    problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] # Sphere
# grab problem 2 
p2 = ioh.get_problem(fid=2, 
                    dimension=n,
                    instance=1,
                    problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] # 
# grab problem 3 
p3 = ioh.get_problem(fid=3, 
                    dimension=n,
                    instance=1,
                    problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #
# grab problem 18 
p18 = ioh.get_problem(fid=18, 
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 23 
p23 = ioh.get_problem(fid=23, 
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 23 
p24 = ioh.get_problem(fid=24, 
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 24
p24 = ioh.get_problem(fid=24,
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue] #

# grab problem 25
p25 = ioh.get_problem(fid=25,
                     dimension=n,
                     instance=1,
                     problem_class=ProblemClass.PBO)  # pyright: ignore[reportCallIssue]

# A list of the whole problems 
Problems: list[ioh.problem.PBO] = [p1, p2, p3, p18, p23, p24, p25]

In [ ]:
# create a list of 3 loggers for each algorithm -- Random Search, RLS (random local search) and (1+1)-EA
root_folder = "my-experiment-notebook"  # pyright: ignore[reportArgumentType]
if os.path.exists(root_folder):
    rmtree(root_folder, ignore_errors=True)

folder_name: str = "run"
algorithm_names: list[str] = ["RandomSearch", "RLS", "OnePlusOneEA"]
# info should be a short description of the algorithm
algorithm_info = [
    "Random Search Algorithm",
    "Random Local Search Algorithm",
    "(1+1)-EA Algorithm"
]

# for name in algorithm_names:

In [30]:
budget = num_iterations = 100000 # budget is the number of function evaluations

In [31]:
num_experiments = 10 # number of independent runs

In [32]:
Loggers: list[logger.Analyzer] = []
# create a list of loggers for each algorithm
for i, name in enumerate(algorithm_names):
    l = logger.Analyzer(
            root=root_folder,  # pyright: ignore[reportArgumentType]
            folder_name=folder_name,
            algorithm_name=name,
            algorithm_info=algorithm_info[i]
        )
    Loggers.append(
       l
    )



In [33]:
L_random_search = Loggers[0]
L_RLS = Loggers[1]
L_OnePlusOneEA = Loggers[2]

#### testing randoms search on problem 1

In [17]:
p1.attach_logger(L_random_search)
algorithm = RandomSearch(budget=budget)
run_experiment(problem=p1, 
            algorithm=algorithm, 
            num_runs=num_experiments)

100.0
run: 1 - best found:  73.000
run: 2 - best found:  72.000
run: 3 - best found:  71.000
run: 4 - best found:  72.000
run: 5 - best found:  70.000
run: 6 - best found:  70.000
run: 7 - best found:  70.000
run: 8 - best found:  71.000
run: 9 - best found:  74.000
run: 10 - best found:  71.000


#### Experiment with the <i>del</i> keyword

In [52]:
L_random_search = Loggers[0]

In [53]:
del L_random_search

In [54]:
Loggers[0]

<Analyzer my-experiment/run>

### Run random search on all our problems (fingers crossed!)

In [35]:
# we can define our random search algorithm once and reuse it thorughout each loop 
random_search_algorithm = RandomSearch(budget=budget)

In [36]:
i = 0
for problem in Problems:
    print(f"================ Begin {i+1} =======================")
    L_random_search = Loggers[0]
    problem.attach_logger(L_random_search)
    # run random search
    run_experiment(problem=problem, 
                algorithm=random_search_algorithm, 
                num_runs=num_experiments)
    print(f"================ End {i + 1} =======================")
    del L_random_search # this is important to avoid logging issues ...
    i += 1

================ Begin 1 =======================
100.0
run: 1 - best found:  70.000
run: 2 - best found:  72.000
run: 3 - best found:  74.000
run: 4 - best found:  70.000
run: 5 - best found:  70.000
run: 6 - best found:  73.000
run: 7 - best found:  71.000
run: 8 - best found:  71.000
run: 9 - best found:  70.000
run: 10 - best found:  72.000
================ End 1 =======================
================ Begin 2 =======================
100.0
run: 1 - best found:  16.000
run: 2 - best found:  20.000
run: 3 - best found:  17.000
run: 4 - best found:  15.000
run: 5 - best found:  16.000
run: 6 - best found:  15.000
run: 7 - best found:  16.000
run: 8 - best found:  16.000
run: 9 - best found:  17.000
run: 10 - best found:  18.000
================ End 2 =======================
================ Begin 3 =======================
5050.0
run: 1 - best found:  3838.000
run: 2 - best found:  3812.000
run: 3 - best found:  3791.000
run: 4 - best found:  3674.000
run: 5 - best found:  3814.000
run

<p>An example of what would happen if we put our log outside instead ... </p>
<p>One can try to guess its behavior before uncommenting and running it.</p>

In [47]:
# i = 0
# L_random_search = Loggers[0]
# for problem in Problems:
#     print(f"================ Begin {i+1} =======================")
#     problem.attach_logger(L_random_search)
#     # run random search
#     run_experiment(problem=problem, 
#                 algorithm=random_search_algorithm, 
#                 num_runs=num_experiments)
#     print(f"================ End {i + 1} =======================")
#     del L_random_search # this is important to avoid logging issues ...
#     i += 1

### <u>Enumerate over all our 3 algorithms</u>

<p>It looks like it worked out pretty well. Now we can include the other 2 algorithms into the mix!</p>

In [18]:
# first, instantiate each algorithm 
rls_algorithm = RandomizedLocalSearch(budget=budget)
one_plus_one_ea_algorithm = OnePlusOneEA(budget=budget)
random_search_algorithm = RandomSearch(budget=budget)


In [34]:
Algorithms = [random_search_algorithm, rls_algorithm]

In [35]:
# runs all 3 algorithms on problem 1
for algorithm in Algorithms:
    print(f"================ Begin {algorithm.name} =======================")
    L = Loggers[algorithm_names.index(algorithm.name)]
    if L is None:
        raise ValueError(f"Logger for algorithm {algorithm.name} not found")
    else: 
        print(f"Successfully found logger for {algorithm.name}")
        input("Press Enter to continue...")
        p1.attach_logger(L)
    
    
    # run random search
    run_experiment(problem=p1, 
                algorithm=algorithm, 
                num_runs=num_experiments)
    print(f"================ End {algorithm.name} =======================")
    del L # this is important to avoid logging issues ...

================ Begin RandomSearch =======================
Successfully found logger for RandomSearch
100.0
run: 1 - best found:  72.000
run: 2 - best found:  72.000
run: 3 - best found:  69.000
run: 4 - best found:  72.000
run: 5 - best found:  70.000
run: 6 - best found:  70.000
run: 7 - best found:  71.000
run: 8 - best found:  73.000
run: 9 - best found:  70.000
run: 10 - best found:  72.000
================ End RandomSearch =======================
================ Begin RLS =======================
Successfully found logger for RLS
100.0
run: 1 - best found:  100.000
run: 2 - best found:  100.000
run: 3 - best found:  100.000
run: 4 - best found:  100.000
run: 5 - best found:  100.000
run: 6 - best found:  100.000
run: 7 - best found:  100.000
run: 8 - best found:  100.000
run: 9 - best found:  100.000
run: 10 - best found:  100.000
================ End RLS =======================


In [ ]:
i = 0
for algorithm in Algorithms:
    print(f"================ Begin Algorithm {i+1} =======================")
    for problem in Problems:
        print(f"========== Begin Problem {problem.meta_data.problem_id} =================")
        L = Loggers[i]
        problem.attach_logger(L)
        
        # run the algorithm
        run_experiment(problem=problem, 
                    algorithm=algorithm, 
                    num_runs=num_experiments)
        del L # this is important to avoid logging issues ...
        print(f"========== End Problem {problem.meta_data.problem_id} =================")
    print(f"=============== End Algorithm {i+1} =======================")
    i += 1

#### Using IOH's built-in Experimental setup 

### Starting off with Random Search

In [ ]:
experiment_1 = ioh.Experiment(
    algorithm = random_search_algorithm,
    algorithm_name = random_search_algorithm.name,
    algorithm_info = random_search_algorithm.algorithm_info,
    fids = [1, 2, 3, 18, 23, 24, 25],
    iids = [1], 
    dims = [100],
    reps = 10, # number of independent repetitions or runs for each problem
    zip_output=True, 
    problem_class=ProblemClass.PBO, # type: ignore
    output_directory="./",
    folder_name=f"ioh-data-{random_search_algorithm.name}-2nd-exp",
    old_logger=False,  # type: ignore
)

In [26]:
experiment_1.run() 

<p>Result from the analyzer (may have to download and zoom for better view) with the y-axis log-scaled</p>

<img src="../final/doc/plots/Random_Search.png"></img>

### With no scaling on both axes

<img src="../final/doc/plots/RandomSearch_No_scaled.png"></img>

### With the x-axis log-scaled 


<img src="../final/doc/plots/Random_Search_x_scaled.png"></img>


#### ><u>Interesting observations from the above set of plots</u>

<ol>
<li>The first observation that we made is that plot 23 and 25 appear to be empty (when the y-axis is scaled). This will be the case for other algorithms as well. This is due to the nature of the problem's objective function's range definition for problems 23 and 25. Both actually started off negative and are maximized to the value 0 over time (function evaluation on the x-axis). This is probably a tell-tale sign that we should stick to applying the log-transmation to the x-axis (or not) only and not to the y-axis. </li>
<li>Now, if we don't apply the log-transformation to the x-axis, we wil end up with the 2nd image above. And it would be difficult to assess the performance of our random search algorithm. But the intriguing part here is that each curve (or each plot) looks almost identical to one another! What could be the reason(s)? One's first instinct could be that it is due to the nature of (pseudo-)randomness of the algorithm itself. The fact that it leads to identical plots speaks more to itself than to the individual problems. Indeed, initially, after some number of function evaluations, the algorithm is able to make one lucky guess that, usually, has a good probability of stumbling upon a solution that is significantly better than the first ones but after that, it spends the next 90-95% of function evaluations trying to find a better one, which some times work with the simpler problems such as OneMax, but not with the non-trivial ones such as F23, F24, and F25 where the upper left corner grows sharper and sharper. In other words, it effectively stagnates for the next 90 - 95% of the total number of evalutations.</li>
<li>After applying the transformation the x-axis, we can see every curve in every plot clearly. We can observe that the algorithm does not perform on the Leading Ones problem. One reason could be that the problem itself requires the <i>first</i> bits (from the left) to be ones and the algorithm most likely discards any progress made in the current iteration and starts over again in the next one.</li>
<li></li>
<li></li>
</ol>

### Now RLS 

In [49]:
experiment_2 = ioh.Experiment(
    algorithm = rls_algorithm,
    algorithm_name = rls_algorithm.name,
    algorithm_info = rls_algorithm.algorithm_info,
    fids = [1, 2, 3, 18, 23, 24, 25],
    iids = [1], 
    dims = [100],
    reps = 10, # number of independent repetitions or runs for each problem
    zip_output=True, 
    problem_class=ProblemClass.PBO, # type: ignore
    output_directory="./",
    folder_name=f"ioh-data-{rls_algorithm.name}",
    old_logger=True,  # type: ignore
)

In [50]:
experiment_2.run()

<img src="../final/doc/plots/RLS.png"></img>

### Finally (1+1)-EA

In [52]:
experiment_3 = ioh.Experiment(
    algorithm = one_plus_one_ea_algorithm,
    algorithm_name = one_plus_one_ea_algorithm.name,
    algorithm_info = one_plus_one_ea_algorithm.algorithm_info,
    fids = [1, 2, 3, 18, 23, 24, 25],
    iids = [1], 
    dims = [100],
    reps = 10, # number of independent repetitions or runs for each problem
    zip_output=True, 
    problem_class=ProblemClass.PBO, # type: ignore
    output_directory="./",
    folder_name= f"ioh-data-{one_plus_one_ea_algorithm.name}",
    old_logger=True,  # type: ignore
)

In [53]:
experiment_3.run()

<img src="../final/doc/plots/OnePlusOneEA.png"></img>

<p>After some more experimental work, the Experimen() class above runs a bit quicker than the loop-based one that I bootstrapped above. But the good news is that both methods work (they practically output the same kind of results that can be pushed into the IOH analyzer!)</p>

It turns out that the IOHAnalyzer is more capable that I had first imagined. It smartly (recursively) walks through output files generated the `Experiment()` class. Below lies the plot results: 

# <b><u>STOP POINT FOR RESULTS</u></b>

## For problem 1: OneMax

### <u>Analyses and explanations of the plots</u>

<p>The analyses consist of informal (but hopefully educated) guesses and judgements based on the problem constraints and parameters. This is still a draft because the finalized version will be stored in .txt files.</p>

<img src="../final/doc/figures/Exercise_2/Scaled_x_y/fid1.png"></img>

- In hindsight, this could be a good draft for the entire 7 plots! The only things missing are some numerical analyses... 

<br>
<p>Let us begin with problem (with fid) 1. It is worth noting that the x-axis has been log-scaled and y-axis stays the same. Although the trend is expected, it is worthwhile to go through it in details (as we should always do). Right away, we can observe that RandomSearch performs the worst out of the other two. Random search basically guesses the solution (randomly) and for the first few number of iterations/function evaluations, it may get lucky and stumble upon a good solution (in this case, one with the highest number of ones). In fact, right after the first evaluation, it achieves a better fitness than RLS. However, from the 10th evaluation onwards, the its random guessing just does not cut it. It gets surpassed by both algorithms by the 30th evaluation. On the other hand, (1+1)-EA initially got an upper hand over RLS until the 20th evaluation which it then gets supplanted and just cannot catch up afterwards. One reason for this is that RLS  is practically perfect for our OneMax problem -- it only accepts improvement after flipping, although random, bit. Whereas, (1+1)-EA attempts to flip all N = 100 bits at a time (with uniform probability 1/n = 0.01), it will inadvertently mis-flip some bits from 1 to 0 early on but as time goes on, it learns to flip less bits, being more careful, and practically behaves like RLS towards the end (hence they converge after the 1000th evaluation).</p>

## For problem 2: 

<img src="../final/doc/figures/Exercise_2/Scaled_x_y/fid2.png"></img>


<u>Explanations:</u>



<p>Like in the first plots, we will see that Random Search exudes very similar behavior to how we described in the case of OneMax. </p>

## Problem 3

<img src="../final/doc/figures/Exercise_2/Scaled_x_y/fid3.png"></img>

## Problem 4

<img src="../final/doc/figures/Exercise_2/Scaled_x_y/fid4.png"></img>

## Problem 18 

<img src="../final/doc/figures/Exercise_2/Scaled_x_y/fid18.png"></img>

## Problem 23 

<img src="../final/doc/figures/Exercise_2/Scaled_x/fid23.png"></img>

## Problem 24 

<img src="../final/doc/figures/Exercise_2/Scaled_x/fid24.png"></img>

## Problem 25

<img src="../final/doc/figures/Exercise_2/Scaled_x/fid25.png"></img>